In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
tf.random.set_seed(777)

print(tf.__version__)

2.6.0


In [3]:
x_data = [[0,0],
          [0,1],
          [1,0],
          [1,1]]
y_data = [[0],
          [1],
          [1],
          [0]]

dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data)).batch(len(x_data))

# float값으로 바꾸기 위해
def preprocess_data(features, labels):
    features = tf.cast(features, tf.float32)
    labels = tf.cast(labels, tf.float32)
    return features, labels

w1 = tf.Variable(tf.random.normal([2,1]), name='weight1')
b1 = tf.Variable(tf.random.normal([1], name = 'bias1'))

w2 = tf.Variable(tf.random.normal([2,1]), name='weight2')
b2 = tf.Variable(tf.random.normal([1], name = 'bias2'))

w3 = tf.Variable(tf.random.normal([2,1]), name='weight3')
b3 = tf.Variable(tf.random.normal([1], name = 'bias3'))

In [5]:
def neural_net(features):
    layer1 = tf.sigmoid(tf.matmul(features, w1) + b1)
    layer2 = tf.sigmoid(tf.matmul(features, w2) + b2)
    layer3 = tf.concat([layer1, layer2], -1)
    layer3 = tf.reshape(layer3, shape = [-1,2])
    hypothesis = tf.sigmoid(tf.matmul(layer3, w3) + b3)
    return hypothesis

def loss_fn(hypothesis, labels):
    cost = -tf.reduce_mean(labels * tf.math.log(hypothesis) + (1-labels) * tf.math.log(1 - hypothesis))
    return cost

optimizer = tf.keras.optimizers.SGD(learning_rate = 0.01)

def accuracy_fn(hypothesis,labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy

def grad(features, labels):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(neural_net(features), labels)
    return tape.gradient(loss_value, [w1,w2,w3,b1,b2,b3])

EPOCHS = 50000
for step in range(EPOCHS):
    for features, labels in dataset:
        features, labels = preprocess_data(features, labels)
        grads = grad(features, labels)
        optimizer.apply_gradients(grads_and_vars = zip(grads, [w1,w2,w3,b1,b2,b3]))
        if step % 5000 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, loss_fn(neural_net(features),labels)))

x_data, y_data = preprocess_data(x_data, y_data)
test_acc = accuracy_fn(neural_net(x_data), y_data)
print("Testset Accuracy: {:.4f}".format(test_acc))


Iter: 0, Loss: 0.8487
Iter: 5000, Loss: 0.6847
Iter: 10000, Loss: 0.6610
Iter: 15000, Loss: 0.6154
Iter: 20000, Loss: 0.5722
Iter: 25000, Loss: 0.5433
Iter: 30000, Loss: 0.5211
Iter: 35000, Loss: 0.4911
Iter: 40000, Loss: 0.4416
Iter: 45000, Loss: 0.3312
Testset Accuracy: 1.0000


In [ ]:
# tf.reduce_mean(x) : 변수 x가 가리키는 배열 전체  원소의 합을 원소 개수로 나누어 계산
# tf.reduce_mean(x, 0)는 열 단위로 평균을 냅니다. 
# tf.reduce_mean(x, 1)는 행 단위로 평균을 냅니다. 